# Course 4 Module 3 Lesson 3 Screencasts

## M3L3SC1: Deploying an LLM Using MLflow and Streamlit Cloud

### Step 1: Setting Up Your Environment

In [ ]:
!pip install mlflow streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 655.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/6

### Step 2: Preparing the Model

In [ ]:
import mlflow
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load your pre-trained model
# model_name = "gpt-3.5-turbo"
model_name = "distilgpt2"
generator = pipeline("text-generation", model=model_name)

class TextGenWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, generator, max_new_tokens=30):
        self.generator = generator
        self.max_new_tokens = max_new_tokens

    def predict(self, context, model_input):
        # `model_input` may be a pandas Series / DataFrame or list-like
        if isinstance(model_input, (pd.Series, pd.DataFrame)):
            prompts = model_input.squeeze().tolist()
        else:
            prompts = model_input
        outputs = self.generator(
            prompts,
            max_new_tokens=self.max_new_tokens,
            do_sample=False
        )
        # Hugging Face returns a list of list of dicts. Extract the text
        return [out[0]["generated_text"] for out in outputs]

pyfunc_model = TextGenWrapper(generator)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


### Step 3: Saving with MLflow

In [ ]:
# Save model with MLflow
mlflow.pyfunc.save_model(
    path="mlflow_model",
    python_model=pyfunc_model
)

2025/06/18 16:33:40 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.21.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torchvision==0.21.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


### Step 4: Creating the User Interface

In [ ]:
import streamlit as st

# Set up the Streamlit interface
st.title("LLM-powered Text Generation")
user_input = st.text_area("Enter your prompt:")

if st.button("Generate"):
    # Load the MLflow model
    model = mlflow.pyfunc.load_model("mlflow_model")
    # Generate a response
    result = model({"text": user_input})
    st.write(result)

2025-06-18 16:33:47.269 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 16:33:47.481 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-18 16:33:47.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 16:33:47.484 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 16:33:47.485 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 16:33:47.486 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 16:33:47.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 16:33:47.488 Thread 'MainThread': mi

In [ ]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.55.114:8501

  Stopping...
  Stopping...
